In [7]:
import torch
import matplotlib.pyplot as plt
import os
import struct
from collections import  defaultdict

from transformers import T5ForConditionalGeneration

In [4]:
def get_exponent(model, nbits):
    exponent = {}
    for idx, (name, param) in enumerate(model.named_parameters()):
        # assert param.dtype is torch.float16
        if param.ndim == 2:
            r, c = param.shape
            tensor_data = list(
                map(lambda x: int(
                        "{}".format(
                                bin(
                                    int.from_bytes(
                                    struct.pack('>e', x), byteorder='big'
                                    )
                                )[3: 3+nbits]      # torch.float16 [3: 8] torch.bfloat16[3: 11]
                            ), 
                            base=2
                    ),
                    param.abs().neg().reshape(-1).tolist()
                )
            )
            tensor = torch.tensor(tensor_data, dtype=torch.int8).reshape(r, c)
            exponent[name] = tensor
    return exponent

In [5]:
### Get the exponent ###
models_hub = {
    "t5": {
        "path": "/home/styaeng/project/delta-compress/pretrained_model/t5",       ### 这里要写成下载后的模型权重文件所在的路径
        "hdlr": T5ForConditionalGeneration.from_pretrained
    }
}
t5_model = models_hub['t5']['hdlr'](models_hub['t5']['path'])
t5_exponent = get_exponent(t5_model, nbits=5)

In [6]:
'''
2 bits
    [0 | 1 | 2 | 3]
3 bits
    [0 | 1 | 2 | 3 | 4 | 5 | 6 | 7]
4 bits
    [0-15]
5 bits
    [0]
'''

pattern = [
    [0, 1, 2, 3],
    [0, 1, 2, 3, 4, 5, 6, 7],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    [0],
]

In [15]:
### Get the percentage in matrix granularity ###
tensor_pattern = []
for k, v in t5_exponent.items():
    # tensor_self_pattern["name"] = k
    tensor_self_pattern = defaultdict(dict)
    tensor_self_pattern['name'] = k
    row, col = v.shape
    t = v
    for shift in range(3, -1, -1):
        idx = 3 - shift
        pat = pattern[idx]
        tmp = t >> shift
        tensor_self_pattern[f'{idx+2}_bits'] = {}
        for elem in pat:
            tensor_self_pattern[f"{idx+2}_bits"][elem] = torch.count_nonzero(tmp == elem) / (row * col)
        # tensor_self_pattern['compress_num'][value] = torch.count_nonzero(t == value) / (row * col)
    tensor_pattern.append(tensor_self_pattern)

In [20]:
### Get the percentage in tile granularity ###
tensor_pattern = []
for k, v in t5_exponent.items():
    # tensor_self_pattern["name"] = k
    tensor_self_pattern = defaultdict(dict)
    # tensor_self_pattern['name']
    row, col = v.shape
    t = v
    for row_idx in range(row):
        tensor_self_pattern['row_id'][row_idx] = {}
        for shift in range(3, -1, -1):
            tile = t[row_idx]
            idx = 3 - shift
            pat = pattern[idx]
            tmp = tile >> shift
            for elem in pat:
                tensor_self_pattern['row_idx'][row_idx][f"{idx+2}_bits"][elem] = torch.count_nonzero(tmp == elem) / col
        tensor_pattern.append(tensor_self_pattern)

KeyError: 0